In [15]:
%run data.ipynb
%run binarizers.ipynb
%run models.ipynb

import torch
from torch.autograd import Variable


In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = Net().to(device)

In [ ]:
epochs = 100
